<a href="https://colab.research.google.com/github/arbouria/Notas-Aprendizaje-y-Comportamiento-Adaptable-I/blob/main/Copia_de_volatilidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Simulador volatilidad

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown

# Función de simulación
def run_simulation(timesteps, change_interval, alpha, lam):
    change_points = np.arange(0, timesteps, change_interval)
    environment = np.zeros(timesteps)
    for i in range(len(change_points)-1):
        start, end = change_points[i], change_points[i+1]
        mean = np.random.uniform(0.2, 0.8)
        var = np.random.uniform(0.05, 0.2)
        intervals = np.random.exponential(scale=1/lam, size=(end-start))
        events = (intervals < 1).astype(float)
        for t in range(start, end):
            prev1 = events[t-start-1] if t-start-1 >= 0 else 0
            prev2 = events[t-start-2] if t-start-2 >= 0 else 0
            hazard = lam * np.exp(-lam * intervals[t-start])
            environment[t] = mean + var * (prev1 + prev2) + hazard * 0.1

    estimate = np.zeros_like(environment)
    estimate[0] = environment[0]
    change_detected = []
    for t in range(1, timesteps):
        estimate[t] = alpha * environment[t] + (1 - alpha) * estimate[t-1]
        if abs(environment[t] - estimate[t-1]) > 2 * np.std(environment[:t]):
            change_detected.append(t)

    # Plot resultados
    plt.figure()
    plt.plot(environment, label='Entorno')
    plt.plot(estimate, label='Estimación')
    plt.scatter(change_detected, environment[change_detected], marker='x', label='Cambios detectados')
    plt.legend()
    plt.xlabel('Paso de tiempo')
    plt.ylabel('Valor')
    plt.show()

# Widgets interactivos
interact(
    run_simulation,
    timesteps=IntSlider(min=200, max=2000, step=100, value=1000, description='Tamaño'),
    change_interval=IntSlider(min=50, max=500, step=50, value=250, description='Cambio cada'),
    alpha=FloatSlider(min=0.01, max=0.5, step=0.01, value=0.1, description='Alpha'),
    lam=FloatSlider(min=0.05, max=1.0, step=0.05, value=0.3, description='Lambda')
)

interactive(children=(IntSlider(value=1000, description='Tamaño', max=2000, min=200, step=100), IntSlider(valu…

<function __main__.run_simulation(timesteps, change_interval, alpha, lam)>